In [1]:
from marubatsu import Marubatsu, Markpat
from collections import defaultdict

def count_markpats(self):
    markpats = defaultdict(int)
 
    # 横方向と縦方向の判定
    for i in range(self.BOARD_SIZE):
        count = self.count_marks(coord=[0, i], dx=1, dy=0, datatype="tuple")
        markpats[count] += 1
        count = self.count_marks(coord=[i, 0], dx=0, dy=1, datatype="tuple")
        markpats[count] += 1
    # 左上から右下方向の判定
    count = self.count_marks(coord=[0, 0], dx=1, dy=1, datatype="tuple")
    markpats[count] += 1
    # 右上から左下方向の判定
    count = self.count_marks(coord=[2, 0], dx=-1, dy=1, datatype="tuple")
    markpats[count] += 1

    return markpats

Marubatsu.count_markpats = count_markpats

In [2]:
from pprint import pprint
mb = Marubatsu()

print(mb)
pprint(mb.count_markpats())

mb.move(1, 1)
print(mb)
pprint(mb.count_markpats())

mb.move(0, 0)
print(mb)
pprint(mb.count_markpats())

mb.move(1, 0)
print(mb)
pprint(mb.count_markpats())

Turn o
...
...
...

defaultdict(<class 'int'>, {Markpat(last_turn=0, turn=0, empty=3): 8})
Turn x
...
.O.
...

defaultdict(<class 'int'>,
            {Markpat(last_turn=0, turn=0, empty=3): 4,
             Markpat(last_turn=1, turn=0, empty=2): 4})
Turn o
X..
.o.
...

defaultdict(<class 'int'>,
            {Markpat(last_turn=0, turn=0, empty=3): 2,
             Markpat(last_turn=0, turn=1, empty=2): 3,
             Markpat(last_turn=1, turn=0, empty=2): 2,
             Markpat(last_turn=1, turn=1, empty=1): 1})
Turn x
xO.
.o.
...

defaultdict(<class 'int'>,
            {Markpat(last_turn=0, turn=0, empty=3): 2,
             Markpat(last_turn=0, turn=1, empty=2): 1,
             Markpat(last_turn=1, turn=0, empty=2): 2,
             Markpat(last_turn=1, turn=1, empty=1): 2,
             Markpat(last_turn=2, turn=0, empty=1): 1})


In [3]:
from ai import ai_by_score

def ai9s(mb, debug=False):
    def eval_func(mb):
        # 真ん中のマスに着手している場合は、評価値として 4 を返す
        if mb.last_move == (1, 1):
            return 4
    
        # 自分が勝利している場合は、評価値として 3 を返す
        if mb.status == mb.last_turn:
            return 3

        markpats = mb.count_markpats()
        # 相手が勝利できる場合は評価値として -1 を返す
        if markpats[Markpat(last_turn=0, turn=2, empty=1)] > 0:
            return -1
        # 次の自分の手番で自分が必ず勝利できる場合は評価値として 2 を返す
        elif markpats[Markpat(last_turn=2, turn=0, empty=1)] >= 2:
            return 2
        # 次の自分の手番で自分が勝利できる場合は評価値として 1 を返す
        elif markpats[Markpat(last_turn=2, turn=0, empty=1)] == 1:
            return 1
        # それ以外の場合は評価値として 0 を返す
        else:
            return 0

    return ai_by_score(mb, eval_func, debug=debug) 

In [4]:
from ai import ai_match, ai2, ai8s

ai_match(ai=[ai9s, ai2])

ai9s VS ai2
count     win    lose    draw
o        9867      15     118
x        8961     237     802
total   18828     252     920

ratio     win    lose    draw
o       98.7%    0.1%    1.2%
x       89.6%    2.4%    8.0%
total   94.1%    1.3%    4.6%



In [5]:
ai_match(ai=[ai9s, ai8s])

ai9s VS ai8s
count     win    lose    draw
o        5019     397    4584
x         409    3365    6226
total    5428    3762   10810

ratio     win    lose    draw
o       50.2%    4.0%   45.8%
x        4.1%   33.7%   62.3%
total   27.1%   18.8%   54.0%



In [6]:
def ai10s(mb, debug=False):
    def eval_func(mb):      
        # 真ん中のマスに着手している場合は、評価値として 12 を返す
        if mb.last_move == (1, 1):
            return 12
    
        # 自分が勝利している場合は、評価値として 11 を返す
        if mb.status == mb.last_turn:
            return 11

        markpats = mb.count_markpats()
        # 相手が勝利できる場合は評価値として -1 を返す
        if markpats[Markpat(last_turn=0, turn=2, empty=1)] > 0:
            return -1
        # 次の自分の手番で自分が必ず勝利できる場合は評価値として 10 を返す
        elif markpats[Markpat(last_turn=2, turn=0, empty=1)] >= 2:
            return 10

        # 評価値の合計を計算する変数を 0 で初期化する
        score = 0        
        # 次の自分の手番で自分が勝利できる場合は評価値に 1 を加算する
        if markpats[Markpat(last_turn=2, turn=0, empty=1)] == 1:
            score += 1
        # 「自 1 敵 0 空 2」の数だけ、評価値を加算する
        score += markpats[Markpat(last_turn=1, turn=0, empty=2)]
        
        # 計算した評価値を返す
        return score

    return ai_by_score(mb, eval_func, debug=debug) 

In [7]:
ai_match(ai=[ai10s, ai2])

ai10s VS ai2
count     win    lose    draw
o        9737       0     263
x        8564     263    1173
total   18301     263    1436

ratio     win    lose    draw
o       97.4%    0.0%    2.6%
x       85.6%    2.6%   11.7%
total   91.5%    1.3%    7.2%



In [8]:
ai_match(ai=[ai10s, ai9s])

ai10s VS ai9s
count     win    lose    draw
o        7478       0    2522
x         791       0    9209
total    8269       0   11731

ratio     win    lose    draw
o       74.8%    0.0%   25.2%
x        7.9%    0.0%   92.1%
total   41.3%    0.0%   58.7%



In [9]:
def ai10s(mb, debug=False):
    def eval_func(mb):      
        # 真ん中のマスに着手している場合は、評価値として 300 を返す
        if mb.last_move == (1, 1):
            return 300
    
        # 自分が勝利している場合は、評価値として 200 を返す
        if mb.status == mb.last_turn:
            return 200

        markpats = mb.count_markpats()
        # 相手が勝利できる場合は評価値として -100 を返す
        if markpats[Markpat(last_turn=0, turn=2, empty=1)] > 0:
            return -100
        # 次の自分の手番で自分が必ず勝利できる場合は評価値として 100 を返す
        elif markpats[Markpat(last_turn=2, turn=0, empty=1)] >= 2:
            return 100

        # 評価値の合計を計算する変数を 0 で初期化する
        score = 0        
        # 次の自分の手番で自分が勝利できる場合は評価値に 5 を加算する
        if markpats[Markpat(last_turn=2, turn=0, empty=1)] == 1:
            score += 5
        # 「自 1 敵 0 空 2」の数だけ、評価値を加算する
        score += markpats[Markpat(last_turn=1, turn=0, empty=2)]
        
        # 計算した評価値を返す
        return score

    return ai_by_score(mb, eval_func, debug=debug)

In [10]:
ai_match(ai=[ai10s, ai2])

ai10s VS ai2
count     win    lose    draw
o        9904       0      96
x        9002     305     693
total   18906     305     789

ratio     win    lose    draw
o       99.0%    0.0%    1.0%
x       90.0%    3.0%    6.9%
total   94.5%    1.5%    3.9%



In [11]:
ai_match(ai=[ai10s, ai9s])

ai10s VS ai9s
count     win    lose    draw
o        4878       0    5122
x         802       0    9198
total    5680       0   14320

ratio     win    lose    draw
o       48.8%    0.0%   51.2%
x        8.0%    0.0%   92.0%
total   28.4%    0.0%   71.6%



次の自分の手番で自分が勝利できる場合は評価値を 1 に戻す

In [29]:
def ai10s(mb, debug=False):
    def eval_func(mb):      
        # 真ん中のマスに着手している場合は、評価値として 300 を返す
        if mb.last_move == (1, 1):
            return 300
    
        # 自分が勝利している場合は、評価値として 200 を返す
        if mb.status == mb.last_turn:
            return 200

        markpats = mb.count_markpats()
        # 相手が勝利できる場合は評価値として -100 を返す
        if markpats[Markpat(last_turn=0, turn=2, empty=1)] > 0:
            return -100
        # 次の自分の手番で自分が必ず勝利できる場合は評価値として 100 を返す
        elif markpats[Markpat(last_turn=2, turn=0, empty=1)] >= 2:
            return 100

        # 評価値の合計を計算する変数を 0 で初期化する
        score = 0        
        # 次の自分の手番で自分が勝利できる場合は評価値に 1 を加算する
        if markpats[Markpat(last_turn=2, turn=0, empty=1)] == 1:
            score += 1
        # 「自 1 敵 0 空 2」の数だけ、評価値を加算する
        score += markpats[Markpat(last_turn=1, turn=0, empty=2)]
        
        # 計算した評価値を返す
        return score

    return ai_by_score(mb, eval_func, debug=debug)

In [12]:
def ai11s(mb, debug=False):
    def eval_func(mb):      
        # 真ん中のマスに着手している場合は、評価値として 300 を返す
        if mb.last_move == (1, 1):
            return 300
    
        # 自分が勝利している場合は、評価値として 200 を返す
        if mb.status == mb.last_turn:
            return 200

        markpats = mb.count_markpats()
        # 相手が勝利できる場合は評価値として -100 を返す
        if markpats[Markpat(last_turn=0, turn=2, empty=1)] > 0:
            return -100
        # 次の自分の手番で自分が必ず勝利できる場合は評価値として 100 を返す
        elif markpats[Markpat(last_turn=2, turn=0, empty=1)] >= 2:
            return 100

        # 評価値の合計を計算する変数を 0 で初期化する
        score = 0        
        # 次の自分の手番で自分が勝利できる場合は評価値に 1 を加算する
        if markpats[Markpat(last_turn=2, turn=0, empty=1)] == 1:
            score += 1
        # 「自 1 敵 0 空 2」の数だけ、評価値を加算する
        score += markpats[Markpat(last_turn=1, turn=0, empty=2)]
        # 「自 0 敵 1 空 2」の数だけ、評価値を減算する
        score -= markpats[Markpat(last_turn=0, turn=1, empty=2)]
        
        # 計算した評価値を返す
        return score

    return ai_by_score(mb, eval_func, debug=debug)

In [13]:
ai_match(ai=[ai11s, ai2])

ai11s VS ai2
count     win    lose    draw
o        9813       0     187
x        8251     193    1556
total   18064     193    1743

ratio     win    lose    draw
o       98.1%    0.0%    1.9%
x       82.5%    1.9%   15.6%
total   90.3%    1.0%    8.7%



In [17]:
ai_match(ai=[ai11s, ai10s])

ai11s VS ai10s
count     win    lose    draw
o        2186       0    7814
x           0    5014    4986
total    2186    5014   12800

ratio     win    lose    draw
o       21.9%    0.0%   78.1%
x        0.0%   50.1%   49.9%
total   10.9%   25.1%   64.0%

